In [1]:
from torch import nn
from transformers import BertModel, BertTokenizer
import json
from typing import List
import os
import numpy as np
import logging
from transformers import AdamW
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import json, sklearn, pickle, sys, re, os
import IPython.display as ipd
import torch
from transformers import *
torch.cuda.empty_cache()

In [2]:
from transformers import AutoTokenizer

In [39]:
datapd=pd.read_json("/home/ubuntu/project/arxiv-metadata-oai-snapshot.json",  lines=True)

In [40]:
datapd_hep_th = datapd.loc[datapd['categories'].str.contains('hep-th')]
hep_abstracts = datapd_hep_th[ list(datapd_hep_th.iloc[:,0:1]) + ['abstract'] + ['title']]

In [41]:
texttxt=hep_abstracts.abstract.values

In [11]:
#!pip install transformers

In [12]:
#!pip install datasets

In [13]:
from datasets import load_dataset
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

In [ ]:
random.shuffle(text_txt)

# The limit 1325 helps makes sure that the max length of a given input is less than 512 #

In [ ]:
g=open("/home/ubuntu/arxiv_small_roberta.txt","w")
count=0
for x in text_txt[0:140000]:
    if len(x)<1325:
        g.write(x)
        g.write("\n")
        count=count+1
g.close()

In [ ]:
g=open("/home/ubuntu/arxiv_dev_roberta.txt","w")
count=0
for x in text_txt[140000:140500]:
    if len(x)<1325:
        g.write(x)
        g.write("\n")
        count=count+1
g.close()

In [14]:
datasets = load_dataset("text", data_files={"train": "/home/ubuntu/project/arxiv_roberta.txt","validation": "/home/ubuntu/project/arxivdev_roberta.txt"})

Using custom data configuration default-5b6716af72ed4b92
Reusing dataset text (/home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57)


In [15]:
print(datasets["train"][0])
print(datasets["validation"][0])

{'text': '  The problem of the gauge dependence of the fermion mass in the Maxwell-Chern-Simons QED$_{2+1}$ is revisited. Using Proca mass term as an intermediate infrared regulator we are demonstrating gauge-invariance of the fermion mass shell in QED$_{2+1}$ in all orders of the perturbation theory. '}
{'text': '  We consider the integer QH state on Riemann surfaces with conical singularities, with the main objective of detecting the effect of the gravitational anomaly directly from the form of the wave function on a singular geometry. We suggest the formula expressing the normalisation factor of the holomorphic state in terms of the regularized zeta determinant on conical surfaces and check this relation for some model geometries. We also comment on possible extensions of this result to the fractional QH states. '}


In [16]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), 
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [17]:
show_random_elements(datasets["train"])

,text
0,"Following the subtraction procedure for manifolds with boundaries, we calculate by variational methods, the Schwarzschild and Flat space energy difference. The one loop approximation for TT tensors is considered here. An analogy between the computed energy difference in momentum space and the Casimir effect is illustrated. We find a singular behaviour in the UV-limit, due to the presence of the horizon when $r=2m.$ When $r>2m$ this singular behaviour disappears, which is in agreement with various other models previously presented."
1,"In the presence of a minimal length physical objects cannot collapse to an infinite density, singular, matter point. In this note we consider the possible final stage of the gravitational collapse of ""thick"" matter layers. The energy momentum tensor we choose to model these shell-like objects is a proper modification of the source for ""non-commutative geometry inspired"", regular black holes. By using higher momenta of Gaussian distribution to localize matter at finite distance from the origin, we obtain new solutions of the Einstein's equation which smoothly interpolates between Minkowski geometry near the center of the shell and Schwarzschild spacetime far away from the matter layer. The metric is curvature singularity free. Black hole type solutions exist only for ""heavy"" shells, i.e. $M\ge M_{e}$, where $M_{e}$ is the mass of the extremal configuration. We determine the Hawking temperature and a modified Area Law taking into account the extended nature of the source."
2,"We holographically investigate the scalarization in the Einstein-Scalar-Gauss-Bonnet gravity with a negative cosmological constant. We find that instability exists for both Schwarzschild-AdS and Reissner-Nordstrom-AdS black holes with planar horizons when we have proper interactions between the scalar field and the Gauss-Bonnet curvature corrections. We relate such instability to possible holographic scalarization and construct the corresponding hairy black hole solutions in the presence of the cosmological constant. Employing the holographic principle we expect that such bulk scalarization corresponds to the boundary description of the scalar hair condensation without breaking any symmetry, and we calculate the related holographic entanglement entropy of the system. Moreover, we compare the mechanisms of the holographic scalarizations caused by the effect of the coupling of the scalar field to the Gauss-Bonnet term and holographic superconductor effect in the presence of an electromagnetic field, and unveil their differences in the effective mass of the scalar field, the temperature dependent property and the optical conductivity."
3,"Several alternative approaches to quantum gravity problem suggest the modification of the {\it fundamental volume $\omega_{0}$} of the accessible phase space for representative points. This modified fundamental volume has a novel momentum dependence. In this paper, we study the effects of this modification on the thermodynamics of an ideal gas within the microcanonical ensemble and using the generalized uncertainty principle(GUP). Although the induced modifications are important only in quantum gravity era, possible experimental manifestation of these effects may provides strong support for underlying quantum gravity proposal."
4,"We present examples of 5d SCFTs that serve as counter-examples to a recently actively studied conjecture according to which it should be possible to obtain all 5d SCFTs by integrating out BPS particles from 6d SCFTs compactified on a circle. We further observe that it is possible to obtain these 5d SCFTs from 6d SCFTs if one allows integrating out BPS strings as well. Based on this observation, we propose a revised version of the conjecture according to which it should be possible to obtain all 5d SCFTs by integrating out both BPS particles and BPS strings from 6d SCFTs compactified on a circle. We describe a general procedure to integrate out BPS string

In [18]:
model_checkpoint = "roberta-base"

In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [23]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [24]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=8, remove_columns=["text"])

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-5b86afd5ccbddfe1.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-8689509a076c9326.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-fd8d4296cf444171.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-ffe6c1bbab5d608d.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-232e0b955f9d44df.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-a99d0adc6e785e0d.arrow


...

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-52f1e9c569aabab5.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-31c7a5c965c51de1.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-ac9f435027f4536b.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-c3b07f825fb3875e.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-a88d8db85aac0f37.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-beca1a65206b55cd.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-40110f2b9645c42c.arrow


In [25]:
tokenized_datasets["train"][1]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'input_ids': [0,
  1437,
  166,
  1455,
  41,
  7740,
  7,
  826,
  4754,
  26305,
  3092,
  6,
  31612,
  106,
  15,
  5,
  1453,
  9,
  49,
  1291,
  19,
  9553,
  4903,
  1538,
  43603,
  3092,
  9,
  17244,
  4,
  166,
  34882,
  5,
  6945,
  3892,
  4621,
  1492,
  8,
  37357,
  17997,
  21349,
  11,
  5,
  29977,
  982,
  1453,
  4,
  1437,
  2]}

In [26]:
block_size = 512

In [27]:
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [28]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=8,
)

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-dfb7bffbed7c0fe8.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-ebd2755534a3e59f.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-ba648e47d4adc52e.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-27606c54d768fbb4.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-148174231f0cf7ec.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-44f51fb51f541155.arrow


...

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-fd17b9259cd874ed.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-d875b426ffbbd9af.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-cc8d7bda312547a5.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-5eb3a9cd9cf3c6b1.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-409e71f47793c36c.arrow


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/text/default-5b6716af72ed4b92/0.0.0/293ecb642f9fca45b44ad1f90c8445c54b9d80b95ab3fca3cfa5e1e3d85d4a57/cache-66cb6ff3433511ba.arrow


In [29]:
tokenizer.decode(lm_datasets["train"][0]["input_ids"])

'<s>  The problem of the gauge dependence of the fermion mass in the Maxwell-Chern-Simons QED$_{2+1}$ is revisited. Using Proca mass term as an intermediate infrared regulator we are demonstrating gauge-invariance of the fermion mass shell in QED$_{2+1}$ in all orders of the perturbation theory. </s><s>  We present an introduction to Group Field Theory models, motivating them on the basis of their relationship with discretized BF models of gravity. We derive the Feynmann rules and compute quantum corrections in the coherent states basis. </s><s>  We study the closed-string spectrum of SU(N) gauge theories in the fundamental representation in 2+1 dimensions. We calculate the energies of the lowest lying ~ 30 states using a large variety of operators characterised by the quantum numbers of parity and longitudinal momentum. We find that our results for the ground state are very well approximated by the Nambu-Goto (NG) predictions even for short strings. For the excited states, we observe 

In [30]:
from transformers import AutoModelForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

In [31]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint,output_hidden_states=True)

In [32]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [33]:
training_args = TrainingArguments(
    "/home/ubuntu/Roberta/robertasave",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    per_device_train_batch_size=4
)

In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"]
)

In [35]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  488367 KB |  488367 KB |  488367 KB |       0 B  |
|       from large pool |  487680 KB |  487680 KB |  487680 KB |       0 B  |
|       from small pool |     687 KB |     687 KB |     687 KB |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |  488367 KB |  488367 KB |  488367 KB |       0 B  |
|       from large pool |  487680 KB |  487680 KB |  487680 KB |

In [36]:
training_args

TrainingArguments(output_dir=/home/ubuntu/Roberta/robertasave, overwrite_output_dir=False, do_train=False, do_eval=None, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Mar14_19-22-13_ip-172-31-36-211, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level=O1, fp16_backend=auto, local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False, debug=False, dataloader_drop_last=False, eval_steps=500, dataloader_num_workers=0, past_index=-1, run_name=/home/ubuntu/Roberta/robertasave, disable_tqdm=False, remove_unused_columns=True, label

In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.000000,0.000001,7.904700,10.500000


TrainOutput(global_step=9619, training_loss=0.003238742601787527, metrics={'train_runtime': 11061.0702, 'train_samples_per_second': 0.87, 'total_flos': 14737871891893248, 'epoch': 1.0})

In [53]:
input_ids = torch.tensor(tokenizer.encode(text_txt[0])).unsqueeze(0)  # Batch size 1

In [54]:
print(input_ids)
device = "cuda:0"
model = model.to(device)
input_ids = input_ids.to(device)
output=model(input_ids)

tensor([[    0,  1437,    20,   936,     9,     5, 12567, 20748,     9,     5,
           856,  8362,  1499,  2862,    11,     5, 15730,    12,   347, 16494,
            12, 29656,  1790,  1209,  1691,  1629, 49747,   176,  2744,   134,
         24303,  1629,    16, 25614,  4560,     4,  8630,  1698,  3245,  2862,
          1385,    25,    41, 21398, 30175,  8199,    52,    32, 16987, 12567,
            12, 24701,  1512,  2389,     9,     5,   856,  8362,  1499,  2862,
         10785,    11,  1209,  1691,  1629, 49747,   176,  2744,   134, 24303,
          1629,    11,    70,  3365,     9,     5, 32819, 13157,  1258,  6680,
             4,  1437,     2]])


In [56]:
model.config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.3.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

#### Let's sample how the output looks like ####

In [77]:
inp = text_txt[15215]
sentence = tokenizer.encode(inp, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
senter=sentence.to(device)
output=model(senter)

tensor([[    0,  1437,    20, 43769,  1076,  1899,  3809,   281,     9,  1402,
          1232,     9, 17997,  6287,  9781,    32,  1687,    11,  4617,     4,
            20, 31625,  7721, 15256,   475,   982,   228,  1082,    36,   119,
         37457,  1899,  1343,   176,   238,    19, 14712,    12,   858,  8774,
         12514, 11324,    19,   121,  1640,   119,    43, 43769,     6,   223,
            61,     5,  3091,  7891, 25935,  7223,   552,     5,  3206,    11,
             5,  6451,   475,     8,    63, 21044,  3252,   877,  8985, 44128,
          4901, 45152,   119, 48634,   166,   465,    14,   209,  6287,  9781,
             6,   190,    19, 49830,   991, 23501, 24303, 48550,     9,   209,
         11324,     6,    33,    10, 43769, 45429,    83,  1215,   119,   203,
          2514,    87,   121,  1640,   119,   238,    61, 19659,    14,     5,
          1007,   364,  8554, 26947,  1136,    88,  4723,    14,    13,   490,
          9781,    36,   118,     4,   242,   482,  

In [78]:
print(output[-1][2].shape)
print(output[-1][11].shape)

torch.Size([1, 512, 768])
torch.Size([1, 512, 768])


In [45]:
iden=hep_abstracts['id'].values
iden[0]

'0704.0015'

In [80]:
torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=None, profile=None, sci_mode=False)

In [81]:
hidden_states=output[1]

In [82]:
last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]

In [83]:
cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
print(cat_hidden_states.size())
cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()
print(cat_sentence_embedding.size())

torch.Size([1, 512, 3072])
torch.Size([3072])


In [84]:
hidden_states[1].shape

torch.Size([1, 512, 768])

In [85]:
hidden_states[0].shape

torch.Size([1, 512, 768])

In [86]:
vecpy=output[-1][0].detach().cpu().numpy()

In [87]:
vecpy.shape

(1, 512, 768)

In [93]:
texttxt[0]

'  The pure spinor formulation of the ten-dimensional superstring leads to\nmanifestly supersymmetric loop amplitudes, expressed as integrals in pure\nspinor superspace. This paper explores different methods to evaluate these\nintegrals and then uses them to calculate the kinematic factors of the one-loop\nand two-loop massless four-point amplitudes involving two and four Ramond\nstates.\n'

In [92]:
iden[0]

'0704.0015'

In [43]:
import os
output_dir = '/home/ubuntu/roberta_model_mar14/'
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/home/ubuntu/roberta_model_mar14/tokenizer_config.json',
 '/home/ubuntu/roberta_model_mar14/special_tokens_map.json',
 '/home/ubuntu/roberta_model_mar14/vocab.json',
 '/home/ubuntu/roberta_model_mar14/merges.txt',
 '/home/ubuntu/roberta_model_mar14/added_tokens.json')

In [8]:
device = "cuda:0"
model = AutoModelForMaskedLM.from_pretrained(output_dir,output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [9]:
model.config

RobertaConfig {
  "_name_or_path": "/home/ubuntu/roberta_model_mar10/",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.3.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [49]:
import time

## We save 6 different kinds of vectors. Zeroth layer, first layer, second layer, last layer, average of last 4 layers and concatenation of last 4 layers ##

In [ ]:
print(time.time())
f = open("mar_14_abstracts_roberta_epoch1_embeddings_v3.txt", "w")
f1 = open("mar_14_abstracts_roberta_epoch1_last4_average_v3.txt", "w")
f2 = open("mar_14_abstracts_roberta_epoch1_last4_concat_v3.txt", "w")
f3 = open("mar_14_abstracts_roberta_epoch1_last_v3.txt", "w")
f4 = open("mar_14_abstracts_roberta_epoch1_first_v3.txt", "w")
f5 = open("mar_14_abstracts_roberta_epoch1_second_v3.txt", "w")
for i in range(100):
    f.write(f"{iden[i]} ")
    f1.write(f"{iden[i]} ")
    f2.write(f"{iden[i]} ")
    f3.write(f"{iden[i]} ")
    f4.write(f"{iden[i]} ")
    f5.write(f"{iden[i]} ")

    sentence = tokenizer.encode(texttxt[i], padding='max_length', max_length=512, truncation=True, return_tensors='pt')
    senter=sentence.to(device)
    output=model(senter)
    hidden_states=output[1]

        
    last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
    cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
    cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()
    vec_cat=cat_sentence_embedding.detach().cpu().numpy()
     
    last_layer = hidden_states[-1] 
    last_sentence_embedding = torch.mean(last_layer, dim=1).squeeze()
    vec_last=last_sentence_embedding.detach().cpu().numpy()
        
    zeroth_layer = hidden_states[0] 
    zeroth_sentence_embedding = torch.mean(zeroth_layer, dim=1).squeeze()
    vec_zero=zeroth_sentence_embedding.detach().cpu().numpy()
    
    first_layer = hidden_states[1] 
    first_sentence_embedding = torch.mean(first_layer, dim=1).squeeze()
    vec_first=first_sentence_embedding.detach().cpu().numpy()
    
    second_layer = hidden_states[2] 
    second_sentence_embedding = torch.mean(second_layer, dim=1).squeeze()
    vec_second=second_sentence_embedding.detach().cpu().numpy()
        
    average_layer=(hidden_states[-1]+hidden_states[-2]+hidden_states[-3]+hidden_states[-4])/4
    avg_sentence_embedding = torch.mean(average_layer, dim=1).squeeze()
    vec_avg=avg_sentence_embedding.detach().cpu().numpy()

    f.write((' '.join(['%0.10f']*vec_zero.size)+'\n') % tuple(vec_zero))
    f1.write((' '.join(['%0.10f']*vec_avg.size)+'\n') % tuple(vec_avg))
    f2.write((' '.join(['%0.10f']*vec_cat.size)+'\n') % tuple(vec_cat))
    f3.write((' '.join(['%0.10f']*vec_last.size)+'\n') % tuple(vec_last))
    f4.write((' '.join(['%0.10f']*vec_last.size)+'\n') % tuple(vec_first))
    f5.write((' '.join(['%0.10f']*vec_last.size)+'\n') % tuple(vec_second))
    if i%1000==0:
        print(i)
f.close()
print(time.time())